In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os

import napari
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.transform import resize
from skimage.exposure import rescale_intensity


In [2]:
dir = "C:/Users/amarx/Desktop/livecell_NRCM/raw"
image_paths = list(Path(dir).rglob("*Membrane*.tif"))

In [3]:
imgs = [io.imread(image_path, as_gray=True) for image_path in image_paths]

In [4]:
imgs = [resize(img, (512, 512)) for img in imgs]

In [5]:
imgs = np.stack(imgs)

In [34]:
from scipy.spatial.distance import cdist

def dtw(source, query):
    n, m = len(source), len(query)
    cost = np.zeros((n+1, m+1))
    cost[:, 0] = np.inf
    cost[0, :] = np.inf
    cost[0, 0] = 0

    cost[1:, 1:] = cdist(source.reshape(-1, 1), query.reshape(-1, 1))

    # adding minimum value to cost
    for i in range(1, n+1):
        for j in range(1, m+1):
            min_val = np.min(
                [cost[i-1, j],
                 cost[i, j-1],
                 cost[i-1, j-1]]
            )
            cost[i, j] = cost[i, j] + min_val

    # traceback path of lowest cost
    source_ixs, query_ixs = [n], [m]
    while (n > 0) or (m > 0):
        tb = np.argmin(
            [cost[n-1, m],
             cost[n, m-1],
             cost[n-1, m-1]]
        )

        if tb == 0:
            n = n - 1
        elif tb == 1:
            m = m - 1
        else:
            n = n - 1
            m = m - 1

        source_ixs.append(n)
        query_ixs.append(m)

    source_ixs.reverse()
    query_ixs.reverse()
    return np.array(source_ixs), np.array(query_ixs)

In [28]:
from skimage.exposure import histogram

img1 = imgs[0, ...]
img2 = imgs[10, ...]

hist1, ixs1 = histogram(img1)
hist2, ixs2 = histogram(img2)

In [35]:
dtw1, dtw2 = dtw(hist1, hist2)

In [37]:
dtw1

array([  0,   1,   2,   3,   3,   4,   5,   5,   6,   7,   7,   8,   9,
        10,  10,  11,  12,  12,  13,  14,  15,  16,  17,  18,  19,  20,
        21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,
        34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
        60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  69,  70,  71,
        72,  72,  72,  73,  74,  75,  76,  77,  78,  79,  80,  80,  81,
        82,  83,  84,  85,  86,  86,  87,  88,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 107, 108, 109, 110, 111, 112, 113, 114, 115, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 145, 145, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 16

In [38]:
dtw2

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  19,  20,  21,  22,  23,  23,
        24,  25,  26,  27,  28,  29,  30,  30,  31,  32,  32,  32,  33,
        34,  34,  35,  35,  36,  37,  38,  38,  39,  40,  41,  41,  42,
        43,  43,  43,  44,  45,  46,  46,  46,  46,  47,  48,  48,  48,
        49,  49,  49,  50,  50,  50,  50,  51,  51,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  59,  59,  60,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  74,  75,  76,
        76,  76,  76,  76,  77,  78,  79,  80,  80,  81,  82,  83,  83,
        84,  85,  86,  87,  88,  89,  90,  90,  90,  91,  91,  92,  93,
        94,  94,  94,  94,  94,  94,  95,  96,  97,  98,  99, 100, 100,
       100, 100, 101, 101, 102, 103, 104, 104, 105, 106, 107, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 115, 116, 117, 118, 118, 118,
       118, 119, 119, 120, 120, 121, 122, 122, 123, 124, 125, 12

In [24]:
dtw(x, y)


(array([4, 3, 2, 1, 1, 1, 0]), array([4, 3, 3, 3, 2, 1, 0]))

In [25]:
napari.view_image(imgs)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 255.5, 255.5), zoom=1.59013671875, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 47.0, 1.0), (0.0, 512.0, 1.0), (0.0, 512.0, 1.0)), current_step=(23, 256, 256), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'imgs' at 0x2469ce36b80>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ]), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_vertic

## Ratio method

In [16]:
from typing import Optional

def ratio_correct(images: np.ndarray, background_intens: Optional[float] = None):
    """
    images: Image stack of shape `N, H, W`.
    """
    # cache image dtype
    dtype = images.dtype

    assert (
        len(images.shape) == 3
    ), f"Expected 3d image stack, instead got {len(images.shape)} dimensions"

    if background_intens is not None:
        assert (
            0 <= background_intens <= 1
        ), f"`background_intens` expected to be between 0 and 1, instead got {background_intens}"

    # calculate the mean intensity for every frame
    # store the intensity from the first frame
    I_mean = np.mean(images, axis=(1, 2))
    I_null = I_mean[0]

    # get the ratio for every frame
    if background_intens is None:
        background_intens = 0
    I_ratio = (I_null - background_intens) / (I_mean - background_intens)

    # subtract background from every pixel
    images = images - background_intens

    # multiply every frame by its ratio
    images = I_ratio.reshape(-1, 1, 1) * images
    return rescale_intensity(images)



In [62]:
t = np.random.randn(5, 10, 2)
t[0, 0, :] = 1


In [72]:
t = t.reshape(5, -1)
hist = np.unique(t, axis=1, return_inverse=True, return_counts=True)

In [74]:
np.unique(t[0, ...], return_counts=True)

(array([-1.99883136, -1.65615604, -1.64384781, -0.94255293, -0.87838359,
        -0.75151743, -0.69459641, -0.50375603, -0.50009564, -0.20078476,
        -0.13146188,  0.27919098,  0.28545618,  0.50507685,  1.        ,
         1.11402096,  1.26564643,  1.31825504,  1.5590659 ]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1],
       dtype=int64))

In [75]:
test = [1, 2, 3, 4, 5]
test[3]

4

In [14]:
t = np.random.randn(10, 8, 2)
np.mean(t, axis=(1, 2)).shape

(10,)